In [1]:
import sys
sys.path.insert(0, '../../') 
from project.data.datasets.ARG_KP_2021.processor import load_dataset
import pandas as pd

### load all arguments with quality score and ground truth kps 

In [2]:
#train_df = pd.read_csv(r"C:\Users\Bilal\crypto\thesis\my-repos\cryptocurrencies-kpa\data\processed\ARG_KP_2021\train_complete.csv", index_col=None)
#dev_df =  pd.read_csv(r"C:\Users\Bilal\crypto\thesis\my-repos\cryptocurrencies-kpa\data\processed\ARG_KP_2021\dev_complete.csv", index_col=None)
test_df =  pd.read_csv(r"C:\Users\Bilal\crypto\thesis\my-repos\cryptocurrencies-kpa\data\processed\ARG_KP_2021\test_complete.csv", index_col=None)

In [3]:
print(len(test_df))
test_df.head(2)

3923


,idx,arg_id,argument,topic,stance,MACE-P,key_point_id,key_point,label,label_strict,label_relaxed,text,sentence1,sentence2
0,0,arg_0_0,Routine child vaccinations isn t mandatory sin...,Routine child vaccinations should be mandatory,-1,0.714195,kp_0_0,Routine child vaccinations or their side effe...,0.0,0.0,0.0,Routine child vaccinations isn t mandatory sin...,[CLS] Routine child vaccinations isn t mandato...,[CLS] Routine child vaccinations or their sid...
1,1,arg_0_0,Routine child vaccinations isn t mandatory sin...,Routine child vaccinations should be mandatory,-1,0.714195,kp_0_1,Mandatory vaccination contradicts basic rights,0.0,0.0,0.0,Routine child vaccinations isn t mandatory sin...,[CLS] Routine child vaccinations isn t mandato...,[CLS] Mandatory vaccination contradicts basic ...


In [4]:
# import the potential kps, was prepare in a separate noteook (prepare_potential_kps.ipnyb)
#all_pot_sent_df = pd.read_csv(r'C:\Users\Bilal\crypto\thesis\my-repos\cryptocurrencies-kpa\data\processed\pot_kps_cleaned.csv', index_col=None)

# try with clustered filtered sentences
all_pot_sent_df = pd.read_csv(r'C:\Users\Bilal\crypto\thesis\my-repos\cryptocurrencies-kpa\data\processed\ARG_KP_2021\candidates_clustering_test.csv', index_col=None)

In [5]:
# to be able to compare, we take only arguments with positive single label
test_ds_labeled = test_df[test_df['label'] == 1]
test_ds_labeled = test_ds_labeled.sample(frac=1).drop_duplicates(subset='arg_id')

### create cross args dataset

Get every possible pair to get matching score from best model. 

In [8]:
# all_pot_sent_df has potential kps sentences and test_ds_labeled has the arguments we want to generate kps for them

def get_paired_args(arguments_df, potential_kps, topic, stance):
    args_ids = []
    args = []
    kps = []
    q_scores = []
    
    filtered_arguments_df = arguments_df[arguments_df['topic'] == topic]
    filtered_potential_kps_df = potential_kps[potential_kps['topic'] == topic]
    
    filtered_arguments_df = filtered_arguments_df[filtered_arguments_df['stance'] == stance]
    filtered_potential_kps_df = filtered_potential_kps_df[filtered_potential_kps_df['stance'] == stance]
    
    unique_args_id = filtered_arguments_df['arg_id'].unique()
    
    for arg_id in unique_args_id:
        arg_row = filtered_arguments_df[filtered_arguments_df["arg_id"] == arg_id].iloc[0]
        for  index, s_row in filtered_potential_kps_df.iterrows():
            if arg_row["argument"] == s_row["sentence"]:
                continue
            args_ids.append(arg_id)
            args.append(arg_row["argument"])
            kps.append(s_row["sentence"])
            q_scores.append(s_row["q_score"])
    
    res_df = pd.DataFrame({
        "arg_id": args_ids,
        "argument": args,
        "key_point": kps,
        "kp_q_score": q_scores
    })
    
    res_df['topic'] = topic
    res_df['stance'] = stance
    return res_df

topics = test_ds_labeled['topic'].unique()
stances = [-1, 1]

cross_pairs_df = None
for topic in topics:
    for stance in stances:
        if cross_pairs_df is None:
            cross_pairs_df = get_paired_args(test_ds_labeled, all_pot_sent_df,  topic, stance)
        else:
            cross_pairs_df = pd.concat([cross_pairs_df, get_paired_args(test_ds_labeled, all_pot_sent_df, topic, stance)])

In [9]:
cross_pairs_df.head(2)

,arg_id,argument,key_point,kp_q_score,topic,stance
0,arg_0_79,child vaccinations shouldn t be mandatory beca...,Routine child vaccinations should not be manda...,0.871068,Routine child vaccinations should be mandatory,-1
1,arg_0_79,child vaccinations shouldn t be mandatory beca...,Children should not be vaccinated because they...,0.865730,Routine child vaccinations should be mandatory,-1


In [10]:
len(cross_pairs_df)

11578

In [40]:
cross_pairs_df.to_csv(r'C:\Users\Bilal\crypto\thesis\my-repos\cryptocurrencies-kpa\data\processed\test_cross_pairs_df.csv', index=False)

### load best model 

Here we load best matcher model and score the pairs

In [41]:
import pandas as pd
test_cross_pairs_df = pd.read_csv(r'C:\Users\Bilal\crypto\thesis\my-repos\cryptocurrencies-kpa\data\processed\test_cross_pairs_df.csv', index_col=None)

In [42]:
len(test_cross_pairs_df)

5539

In [43]:
test_cross_pairs_df['idx'] = test_cross_pairs_df.index
test_cross_pairs_df.head(2)

,arg_id,argument,key_point,kp_q_score,topic,stance,idx
0,arg_1_76,it is dangerous because the government will be tempted to use it to its advantage,Government regulation of social media would be harmful to democracy,0.910105,Social media platforms should be regulated by the government,-1,0
1,arg_1_76,it is dangerous because the government will be tempted to use it to its advantage,they are a privates companies and shouldn't have to be regulated,0.757534,Social media platforms should be regulated by the government,-1,1


In [44]:
# process data to work with the classifier
from project.data.datasets.ARG_KP_2021.processor import preprocessing

test_cross_pairs_df['argument'] = test_cross_pairs_df['argument'].apply(lambda x: preprocessing(x))
test_cross_pairs_df['key_point'] = test_cross_pairs_df['key_point'].apply(lambda x: preprocessing(x))
test_cross_pairs_df['topic'] = test_cross_pairs_df['topic'].apply(lambda x: preprocessing(x))

test_cross_pairs_df['sentence1'] = '[CLS] ' + test_cross_pairs_df['argument'] + ' [SEP] ' + test_cross_pairs_df['topic'] + ' [SEP]'
test_cross_pairs_df['sentence2'] = '[CLS] ' + test_cross_pairs_df['key_point'] + ' [SEP] ' + test_cross_pairs_df['topic'] + ' [SEP]'

In [45]:
test_cross_pairs_df.to_csv(r'C:\Users\Bilal\crypto\thesis\my-repos\cryptocurrencies-kpa\data\processed\test_cross_pairs_clean_df.csv', index=False)

In [46]:
# load best model
import torch
torch.cuda.empty_cache()
from project.models.baselines.classification_model import ClassificationModel
from project.models.common.nlp_configs import HFTransformerDataConfig
from transformers import AutoTokenizer
from project.data.nlp_data_module import NLPDataModule

test_data_path = r'C:\Users\Bilal\crypto\thesis\my-repos\cryptocurrencies-kpa\data\processed\test_cross_pairs_clean_df.csv'
model_checkpoint_path = r"C:\Users\Bilal\crypto\thesis\my-repos\cryptocurrencies-kpa\models\good-models\epoch=1-step=807.ckpt"
model = ClassificationModel.load_from_checkpoint(checkpoint_path=model_checkpoint_path)
data_cfg = HFTransformerDataConfig(**{
    "batch_size": 32,
    "num_workers": 4,
    "train_file": test_data_path,
    "test_file": test_data_path
})
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model.model.config._name_or_path, use_fast=True)
datamodule = NLPDataModule(cfg=data_cfg, tokenizer=tokenizer)
datamodule.setup("test")
test_dataset = datamodule.ds["test"]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Using custom data configuration default-fdcbb798e14496b6


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to C:\Users\Bilal\.cache\huggingface\datasets\csv\default-fdcbb798e14496b6\0.0.0\bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/6 [00:00<?, ?ba/s]

In [47]:
test_dataset.column_names

['arg_id',
 'argument',
 'attention_mask',
 'attention_mask_sent1',
 'attention_mask_sent2',
 'idx',
 'input_ids',
 'input_ids_sent1',
 'input_ids_sent2',
 'key_point',
 'kp_q_score',
 'sentence1',
 'sentence2',
 'stance',
 'topic']

In [48]:
# score match score for every possible pair
BATCH_SIZE = 32

test_ds_df = test_dataset.to_pandas()
test_ds_df.set_index('idx')
test_ds_df['score'] = -1

model.to('cuda')

def add_preds(examples):
    similarity, _, output = model._forward(examples, predict_with_gpu=True)
    for i, idx in enumerate(examples["idx"]):
        if output is not None:
            score = output[i][1].item()
        else:
            score = similarity[i].item()
        test_ds_df.iat[idx.item(), test_ds_df.columns.get_loc('score')] = score

test_dataset.map(lambda examples: add_preds(examples), batched=True, batch_size=BATCH_SIZE)



  0%|          | 0/174 [00:00<?, ?ba/s]

Dataset({
    features: ['arg_id', 'argument', 'attention_mask', 'attention_mask_sent1', 'attention_mask_sent2', 'idx', 'input_ids', 'input_ids_sent1', 'input_ids_sent2', 'key_point', 'kp_q_score', 'sentence1', 'sentence2', 'stance', 'topic'],
    num_rows: 5539
})

In [49]:
test_ds_df.head(2)
test_ds_df.to_csv(r'C:\Users\Bilal\crypto\thesis\my-repos\cryptocurrencies-kpa\data\processed\test_cross_pairs_with_match_score.csv', index=False)

In [2]:
import pandas as pd
test_ds_df = pd.read_csv(r'C:\Users\Bilal\crypto\thesis\my-repos\cryptocurrencies-kpa\data\processed\test_cross_pairs_with_match_score.csv', index_col=None)

In [3]:
print(len(test_ds_df))
test_ds_df.head(1)

5539


,arg_id,argument,attention_mask,attention_mask_sent1,attention_mask_sent2,idx,input_ids,input_ids_sent1,input_ids_sent2,key_point,kp_q_score,sentence1,sentence2,stance,topic,score
0,arg_1_76,it is dangerous because the government will be...,[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1...,[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1...,[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1...,0,[ 0 10975 7454 104 742 24 16 27...,[ 0 10975 7454 104 742 24 16 27...,[ 0 10975 7454 104 742 1621 5746 ...,Government regulation of social media would be...,0.910105,[CLS] it is dangerous because the government w...,[CLS] Government regulation of social media wo...,-1,Social media platforms should be regulated by ...,0.491867


In [5]:
test_ds_df[['sentence1', 'sentence2', 'stance', 'topic', 'score']][test_ds_df['score']> 0.5].sort_values(by='score', ascending=True).head(2)

,sentence1,sentence2,stance,topic,score
155,[CLS] Government regulation of social media wo...,[CLS] This deprives citizens to receive differ...,-1,Social media platforms should be regulated by ...,0.500039
986,[CLS] Governments should avoid the proliferati...,[CLS] The advantage of this is that many false...,1,Social media platforms should be regulated by ...,0.500077


In [6]:
# get top kps based on their number of matches (based on a threshold)
threshold = 0.5
result_df = test_ds_df[test_ds_df['score'] > threshold]
# choose best possible kp considering quality + match + rule-based
# first for every arg_id, get top 20 matches
# result_df = test_ds_df.groupby(by="arg_id").apply(lambda df: df.sample(frac=1, random_state=42).sort_values(by="score", ascending=False).head(1)).reset_index(drop=True)
# add combined average score quality+match/2 
# result_df['combined_score'] = (result_df['kp_q_score'] + result_df['score'] )/ 2
# second take the arg with top combined score from the 20 matches
# result_df = result_df.groupby(by="arg_id").apply(lambda df: df.sample(frac=1, random_state=42).sort_values(by="combined_score", ascending=False).head(1)).reset_index(drop=True)

In [7]:
len(result_df)

1171

In [8]:
print(len(result_df))
result_df.head(2)

1171


,arg_id,argument,attention_mask,attention_mask_sent1,attention_mask_sent2,idx,input_ids,input_ids_sent1,input_ids_sent2,key_point,kp_q_score,sentence1,sentence2,stance,topic,score
4,arg_1_76,it is dangerous because the government will be...,[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1...,[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1...,[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1...,4,[ 0 10975 7454 104 742 24 16 27...,[ 0 10975 7454 104 742 24 16 27...,[ 0 10975 7454 104 742 5 168 ...,the government is corrupt and could take advan...,0.813914,[CLS] it is dangerous because the government w...,[CLS] the government is corrupt and could take...,-1,Social media platforms should be regulated by ...,0.576018
9,arg_1_76,it is dangerous because the government will be...,[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1...,[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1...,[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1...,9,[ 0 10975 7454 104 742 24 16 27...,[ 0 10975 7454 104 742 24 16 27...,[ 0 10975 7454 104 742 125 98 ...,But so also does the potential for exploitation,0.662973,[CLS] it is dangerous because the government w...,[CLS] But so also does the potential for explo...,-1,Social media platforms should be regulated by ...,0.710415


In [16]:
# get the top 10 most frequent KPs 

def get_topic_kps_counts(df):
    unique_topics = df['topic'].unique()
    unique_stances = [-1, 1]
    
    topics = []
    stances = []
    kps = []
    counts = []
    q_scores = []
    for t in unique_topics:
        for stance in unique_stances:
            f_df = df[df['topic'] == t]
            f_df = f_df[f_df['stance'] == stance]   
            unique_kps = f_df['key_point'].unique()
           
            for kp in unique_kps:
                topics.append(t)
                stances.append(stance)
                kps.append(kp)
                #q_scores.append(f_df[f_df['key_point'] == kp].iloc[0]['kp_q_score'])
                counts.append(len(f_df[f_df['key_point'] == kp]))

    return pd.DataFrame({
            "topic": topics,
            "stance": stances,
            "key_point": kps,
            #"q_score": q_scores,
            "counts": counts
        })
    

In [11]:
counts_df = get_topic_kps_counts(result_df)

In [12]:
counts_df.head(2)

,topic,stance,key_point,q_score,counts
0,Social media platforms should be regulated by ...,-1,the government is corrupt and could take advan...,0.813914,5
1,Social media platforms should be regulated by ...,-1,But so also does the potential for exploitation,0.662973,12


In [13]:
counst_df = counts_df.groupby(by=["topic", "stance"]).apply(lambda df: df.sample(frac=1, random_state=42).sort_values(by=["counts", "q_score"], ascending=(False, False)).head(5)).reset_index(drop=True)

In [14]:
len(counst_df)

24

In [15]:
pd.set_option('display.max_colwidth', None)
counst_df.head(24)

,topic,stance,key_point,q_score,counts
0,Routine child vaccinations should be mandatory,-1,could be damaged by side effects I honestly don t consider it safe,0.643530,54
1,Routine child vaccinations should be mandatory,-1,They should not be mandatory it is a personal decision,0.717764,38
2,Routine child vaccinations should be mandatory,-1,As long as vaccines are not free of side effects it cannot make them mandatory for our children,0.862094,37
3,Routine child vaccinations should be mandatory,-1,No the free will of the people should be respected,0.611934,32
4,Routine child vaccinations should be mandatory,1,Yes it is a way to guarantee health,0.637232,124
5,Routine child vaccinations should be mandatory,1,Child vaccinations should be mandatory to prevent children becoming seriously ill from diseases and spreading infections,0.930479,94
6,Routine child vaccinations should be mandatory,1,taking into account that the virus is very contagious it should be mandatory to avoid future mutations,0.822368,57
7,Routine child vaccinations should be mandatory,1,Yes it is necessary to preserve the life of the child,0.683039,35
8,Social media platforms should be regulated by the government,-1,It is against freedom of expression,0.811593,64
9,Social media platforms should be regulated by the government,-1,Government regulation of social media would be harmful to democracy,0.910105,49


In [ ]:
import pandas as pd
test_ds_labeled_counts = pd.read_csv(r"C:\Users\Bilal\crypto\thesis\my-repos\cryptocurrencies-kpa\data\processed\test_ds_labeled_counts.csv", index_col=None)

In [26]:
test_ds_labeled_counts.head(40)

,topic,stance,key_point,counts
0,Routine child vaccinations should be mandatory,-1,Routine child vaccinations or their side effects are dangerous,53
1,Routine child vaccinations should be mandatory,-1,The parents and not the state should decide,23
2,Routine child vaccinations should be mandatory,-1,Mandatory vaccination contradicts basic rights,18
3,Routine child vaccinations should be mandatory,-1,Routine child vaccinations are not necessary to keep children healthy,12
4,Routine child vaccinations should be mandatory,1,Children should not suffer from preventable diseases,46
5,Routine child vaccinations should be mandatory,1,Routine child vaccinations should be mandatory to prevent virus disease spreading,43
6,Routine child vaccinations should be mandatory,1,Routine child vaccinations are necessary to protect others,21
7,Routine child vaccinations should be mandatory,1,Routine child vaccinations are effective,19
8,Social media platforms should be regulated by the government,-1,Social media regulation harm freedom of speech and other democratic rights,50
9,Social media platforms should be regulated by the government,-1,Social media regulation harms privacy,15


# compare with the ground truth
 now we should compare result_df with test_ds_labeled , each having 500 arg_id

In [27]:
import rouge
# evaluate to ground truth using rouge
evaluator = rouge.Rouge(metrics=['rouge-l'],
                           limit_length=True,
                           length_limit_type='words',
                           alpha=0.5, # Default F1_score
                           weight_factor=1.2,
                           stemming=True)

In [31]:
# comparing test_ds_labeled_counts with counst_df
# for every topic, stance => compare predicted kp with every labeled kp and take max score
import numpy as np
from itertools import product
unique_topics = test_ds_labeled['topic'].unique()
unique_stances = [-1, 1]

topics = []
stances = []
scores = []

for t in unique_topics:
    for s in unique_stances:
        test_df = test_ds_labeled_counts[test_ds_labeled_counts['topic'] == t]
        test_df = test_df[test_df['stance'] == s]
        res_df = counst_df[counst_df['topic'] == t]
        res_df = res_df[res_df['stance'] == s]
        
        unique_predicted_kps = res_df['key_point'].unique()
        unique_labeled_kps = test_df['key_point'].unique()
        kp_scores = []
        for p_kp, l_kp in product(unique_predicted_kps, unique_labeled_kps) :
                score = evaluator.get_scores(p_kp, l_kp)
                kp_scores.append(score['rouge-l']['f'])    

        topics.append(t)
        stances.append(s)
        scores.append(np.mean(kp_scores))
        
    
results_df = pd.DataFrame({"topic": topics, "stance": stances,  "r_scores": scores})
    

In [32]:
results_df.head(6)

,topic,stance,r_scores
0,The USA is a good country to live in,-1,0.187573
1,The USA is a good country to live in,1,0.146160
2,Routine child vaccinations should be mandatory,-1,0.127059
3,Routine child vaccinations should be mandatory,1,0.211918
4,Social media platforms should be regulated by the government,-1,0.164286
5,Social media platforms should be regulated by the government,1,0.182226


In [99]:
# key points in labeled ds are completely new and not another arguments like the way we do it, difficult to get good rouge score with our method.
test_ds_labeled[test_ds_labeled['argument'] == test_ds_labeled[test_ds_labeled['arg_id'] == 'arg_0_99']['key_point'].iloc[0]]

,idx,arg_id,argument,topic,stance,MACE-P,key_point_id,label,key_point,text,sentence1,sentence2,subset
